In [1]:
from ultralytics import YOLO
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\minsub\.conda\envs\bax\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사람의 바운딩 박스 크기로 판별하는 함수
def person_size(results, checking_class): # 욜로 모델로 예측된 결과값과 각 객체들의 예측 클래스 번호가 담긴 리스트를 인자로 받음
    for result in results: # 검출된 객체들을 순회
        boxes = result.boxes # 각 객체의 bbox정보를 boxes에 할당

    persons =[] # 객체들이 담긴 리스트에서 사람으로 판별된 객체 리스트의 인덱스 번호를 담을 리스트
    bbox_sizes = [] # 바운딩 박스의 크기 정보를 담을 리스트

    for idx, data in enumerate(checking_class): # 탐지된 객체들의 클래스 정보가 담긴 리스트 
        if data == 4:  # 클래스 번호가 6일시 ("사람"클래스의 번호가 6임)
            person_num = idx # 해당 리스트의 인덱스 번호를 persons리스트에 추가 
            persons.append(person_num)
            
    print(persons)
    # 사람으로 검출된 객체들의 바운딩 박스 넓이를 구해서 리스트에 추가하는 작업
    for person_num in persons: # 사람으로 탐지된 객체의 인덱스 번호가 담긴 리스트를 순회
        if person_num != -1:
            person_xyxy_cordi = boxes.xyxy[person_num].tolist() # 해당 객체의 바운딩 박스의 좌상단 우하단 좌표
            bbox_size = (person_xyxy_cordi[2] - person_xyxy_cordi[0]) * (person_xyxy_cordi[3] - person_xyxy_cordi[1]) # 좌표를 통해 바운딩 박스의 가로 세로 길이를 구해 넓이를 구함
            bbox_sizes.append(bbox_size) # 구한 바운딩 박스 넓이를 리스트에 추가

    # 가장 큰 바운딩 박스의 인덱스 찾기
    largest_bbox_idx = bbox_sizes.index(max(bbox_sizes))

    print("가장 큰 바운딩 박스의 인덱스:", largest_bbox_idx)

    person_num = persons[largest_bbox_idx] # 가장 넓이가 큰 바운딩 박스의 인덱스 번호로 정보 반환
    person_num

    eye_yn, leg_yn, loc, mouth_yn, size, arm_yn = person_assess(person_num, results, checking_class) # 가장큰 사람의 정보를 받아 각 요소들 라벨링하는 함수

    return eye_yn, leg_yn, loc, mouth_yn, size, arm_yn

In [3]:
# 검출된 객체간의 iou를 계산하는 함수
def cal_iou(boxA, boxB):
    
    # boxA와 boxB의 x, y 좌표 값 중 큰 값을 선택하여 IoU 계산에 사용할 시작 좌표를 결정
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    
    # boxA와 boxB의 x, y 좌표 값 중 작은 값을 선택하여 IoU 계산에 사용할 끝 좌표를 결정
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # 두 상자의 겹치는 영역의 넓이 계산
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 각 상자의 전체 영역의 넓이 계산
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # iou계산
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [4]:
# 가장 큰 bbox 사람의 인덱스 번호, 객체들 정보, 객체들의 클래스 리스트를 받아 각 요소들을 라벨링하는 함수
def person_assess(person_num, results, checking_class):
    for result in results: # 탐지된 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox정보를 boxes에 할당

    person_xyxy_cordi = boxes.xyxy[person_num].tolist() # 바운딩 박스의 좌상단 우하단 좌표 리스트
    person_center_x_cordi = boxes.xywh[person_num].tolist()[0] # 바운딩 박스의 중심좌표와 가로 세로 길이 리스트

    # 눈, 팔, 다리, 입, 위치, 크기
    
    # 눈 ============================================================
    eye_class_num = [] # 눈 클래스의 인덱스 번호를 담을 리스트
    for idx, data in enumerate(checking_class): # 객체들의 클래스 번호가 담긴 리스트를 순회하면서
        if data == 1: # 리스트에 담긴 클래스 번호가 1 일시
            eye_class_num.append(idx) # 해당 인덱스 번호를 eye_class_num에 추가

    eye_xyxy_cordi = [] # 눈 바운딩 박스의 좌표 정보를 담을 리스트
    for idx in eye_class_num: # 눈의 인덱스 번호가 담긴 리스트를 순회 하면서
        eye_xyxy_cordi.append(boxes.xyxy[idx].tolist()) # 눈의 바운딩 박스 좌상단 우하단 좌표 리스트를 리스트에 추가

    eye_iou = [] # 눈의 iou 결과값을 담을 리스트
    for i in eye_xyxy_cordi: # 눈의 바운딩 박스 정보가 담긴 리스트를 순회
        eye_iou.append(cal_iou(person_xyxy_cordi,i)) # 사람 바운딩 박스와 눈 바운딩 박스의 iou를 계산한 값을 리스트에 추가

    eye_count = 0 # 눈의 개수 초기화
    for i in eye_iou: # 눈와 사람의 iou값을 순회
        if i != 0:# iou값이 0이 아니면(iou가 0일시 사람와 눈가 겹치지않음 -> 사람로 판단된 영역안에 눈가 없기에 눈로 판단된 객체가 잘못 된것이라 예측해 카운팅X)
            eye_count += 1 # 눈의 개수 +1

    eye_yn = ''  # 눈 라벨링 정보 초기화
    if eye_count == 0: # 눈의 카운팅수가 0일시 
        eye_yn = 'n' # 눈의 라벨링을 'n'으로 없다라고 판정
    else:
        eye_yn = 'y' # 눈의 라벨링을 'y'으로 있다라고 판정

    # 위와 같은 과정으로 나머지 요소들도 iou를 바탕으로 사람로 판정된 영역 내에 겹치지 않는 것을 제외하고 카운팅해 라벨링
    # 이후 요소들의 주석은 눈과 똑같기에 생략 
    
    # 팔 ============================================================
    arm_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 0:
            arm_class_num.append(idx)

    arm_xyxy_cordi = []
    for idx in arm_class_num:
        arm_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    arm_iou = []
    for i in arm_xyxy_cordi:
        arm_iou.append(cal_iou(person_xyxy_cordi,i))

    arm_count = 0
    for i in arm_iou:
        if i != 0:
            arm_count += 1

    arm_yn = ''
    if arm_count == 0:
        arm_yn = 'n'
    else:
        arm_yn = 'y'

    # 다리 ============================================================
    leg_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 2:
            leg_class_num.append(idx)

    leg_xyxy_cordi = []
    for idx in leg_class_num:
        leg_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    leg_iou = []
    for i in leg_xyxy_cordi:
        leg_iou.append(cal_iou(person_xyxy_cordi,i))

    leg_count = 0
    for i in leg_iou:
        if i != 0:
            leg_count += 1

    leg_yn = ''
    if leg_count == 0:
        leg_yn = 'n'
    else:
        leg_yn = 'y'

    # 입 ============================================================
    mouth_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 3:
            mouth_class_num.append(idx)

    mouth_xyxy_cordi = []
    for idx in mouth_class_num:
        mouth_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    mouth_iou = []
    for i in mouth_xyxy_cordi:
        mouth_iou.append(cal_iou(person_xyxy_cordi,i))

    mouth_count = 0
    for i in mouth_iou:
        if i != 0:
            mouth_count += 1

    mouth_yn = ''
    if mouth_count == 0:
        mouth_yn = 'n'
    else:
        mouth_yn = 'y'

    image_w = boxes.orig_shape[0] * 0.965
    image_h = boxes.orig_shape[1]
    image_size = image_w * image_h
    person_size = (person_xyxy_cordi[2] - person_xyxy_cordi[0]) * (person_xyxy_cordi[3] - person_xyxy_cordi[1])
    person_size_percent = person_size / image_size * 100
    size = ''
    if person_size_percent < 16 * 0.8:
        size = 'small'
    elif person_size_percent >= 16 * 0.8 and person_size_percent < 40 * 0.8:
        size = 'middle'
    elif person_size_percent >= 40 * 0.8:
        size = 'big'
    loc = ''
    if person_center_x_cordi < image_w/3:
        loc = 'left'
    elif person_center_x_cordi >= image_w/3 and person_center_x_cordi < image_w/3*2:
        loc = 'center'
    elif person_center_x_cordi >= image_w/3*2:
        loc = 'right'
    
    return eye_yn, leg_yn, loc, mouth_yn, size, arm_yn

In [12]:
# 사람로 탐지된 객체수에 따라 라벨링을 결정
def person_csv(results): # 객체들의 정보를 인자로 받음
    for result in results: # 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox 정보를 boxes로 할당
    
    checking_class = boxes.cls.int().tolist() # 각 bbox의 클래스 번호를 리스트로 반환

    person_num = -1 # 사람 인덱스 번호 초기화
    
    for idx, data in enumerate(checking_class): # 클래스 번호가 담긴 리스트를 순회하며
        if data == 4: # 클래스 번호가 4일시 (사람 클래스 번호가 4)
            person_num = idx # 해당 인덱스 번호를 할당후 for문 탈출
            break
    
    if person_num != -1: # 사람 인덱스 번호가 -1이 아닐시 (사람로 검출된 객체가 있음)
        if checking_class.count(4) == 0: # 객체들의 클래스 번호가 담긴 리스트에서 4이 0개 일시
            print('사람이 검출되지 않았습니다.')
        else:
            # 크기가 가장 큰 바운딩 박스 찾는 함수 호출
            eye_yn, leg_yn, loc, mouth_yn, size, arm_yn = person_size(results, checking_class)
    else: # 오류 방지 차원 알고리즘으로 라벨링 불가시
        eye_yn = 'no detect'
        leg_yn = 'no detect'
        mouth_yn = 'no detect'
        arm_yn = 'no detect'
        loc = 'no detect'
        size = 'no detect'

    return eye_yn, leg_yn, loc, mouth_yn, size, arm_yn

In [7]:
model = YOLO('./person_x_weight/weights/best.pt') # 모델 로드

In [8]:
def yolov8_person_detect(path,filename): # 검사할 이미지의 경로와 파일 이름을 인자로
    
    results = model.predict(path+ '/' + filename) # 로드한 모델로부터 예측결과를 results에 할당
    
    res_plot = results[0].plot()
    
    plt.imshow(res_plot)

    eye_yn, leg_yn, loc, mouth_yn, size, arm_yn = person_csv(results)
    
    return eye_yn, leg_yn, loc, mouth_yn, size, arm_yn

In [10]:
def person_save(img_dir, save_dir):
    
    all_images = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f)) and f.endswith(('.png', '.jpg', '.jpeg'))]

    filenames = []

    person_df = pd.DataFrame(columns=('id', 'eye_yn', 'leg_yn', 'loc', 'mouth_yn', 'size', 'arm_yn'))

    for i in all_images:

        filenames.append(i.split('.')[0][:-4])

    for filename, i in tqdm(zip(all_images, filenames)):
        eye_yn, leg_yn, loc, mouth_yn, size, arm_yn = yolov8_person_detect(img_dir, filename)
        output_filename =save_dir + i + '.jpg'
        plt.savefig(output_filename, format='jpg', dpi=300)
        plt.close()
        person_df.loc[len(person_df)] = [i, eye_yn, leg_yn, loc, mouth_yn, size, arm_yn] # 행기준으로 추가
    
    person_df.to_csv(save_dir +'person_result2.csv', index=False) # csv파일로 저장

In [13]:
person_save(img_dir='./person.v1i.yolov8/test/images', save_dir= './person_result/')

0it [00:00, ?it/s]
image 1/1 c:\yolov8\person.v1i.yolov8\test\images\24_175_23006_person_jpg.rf.690f68c11a390bc87763bb6774dfada5.jpg: 640x640 2 arm_yns, 2 eye_yns, 2 leg_yns, 1 mouth_yn, 1 person, 181.8ms
Speed: 4.0ms preprocess, 181.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
1it [00:00,  2.70it/s]


[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\person.v1i.yolov8\test\images\24_175_23007_person_jpg.rf.44e75451099fc23f6bc9f6d2fbce183b.jpg: 640x640 3 leg_yns, 2 persons, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
2it [00:00,  3.38it/s]


[2, 3]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\person.v1i.yolov8\test\images\24_175_23015_person_jpg.rf.6a959ab6094579e28afe904fdfe35fb9.jpg: 640x640 2 arm_yns, 2 eye_yns, 2 leg_yns, 1 mouth_yn, 1 person, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
3it [00:00,  3.77it/s]


...

[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\person.v1i.yolov8\test\images\5_29_23034_person_jpg.rf.d1ac173b36f84babc809576bd5abeaa8.jpg: 640x640 1 arm_yn, 2 eye_yns, 2 leg_yns, 1 mouth_yn, 1 person, 59.0ms
Speed: 1.0ms preprocess, 59.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
98it [00:23,  4.26it/s]


[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\person.v1i.yolov8\test\images\5_29_23035_person_jpg.rf.fc2a47fb1203dcd10454cdcc501dc3e9.jpg: 640x640 2 arm_yns, 3 eye_yns, 2 leg_yns, 1 mouth_yn, 1 person, 57.0ms
Speed: 1.0ms preprocess, 57.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
99it [00:24,  4.12it/s]

[0]
가장 큰 바운딩 박스의 인덱스: 0


In [14]:
person_ori_df = pd.read_csv('./dbi_person.csv')
person_pred_df = pd.read_csv('./person_result/person_result2.csv')
person_ori_100_df = person_ori_df[person_ori_df['id'].isin(person_pred_df['id'])]

eye_count = 0
leg_count = 0
mouth_count = 0
arm_count = 0
loc_count = 0
size_count = 0

for i in range(len(person_pred_df)):
    if person_ori_100_df['eye_yn'].tolist()[i] != person_pred_df['eye_yn'].tolist()[i]:
        eye_count += 1
    if person_ori_100_df['leg_yn'].tolist()[i] != person_pred_df['leg_yn'].tolist()[i]:
        leg_count += 1
    if person_ori_100_df['mouth_yn'].tolist()[i] != person_pred_df['mouth_yn'].tolist()[i]:
        mouth_count += 1
    if person_ori_100_df['arm_yn'].tolist()[i] != person_pred_df['arm_yn'].tolist()[i]:
        arm_count += 1
    if person_ori_100_df['loc'].tolist()[i] != person_pred_df['loc'].tolist()[i]:
        loc_count += 1
    if person_ori_100_df['size'].tolist()[i] != person_pred_df['size'].tolist()[i]:
        size_count += 1

print(f'eye diff : {eye_count}')
print(f'leg diff : {leg_count}')
print(f'mouth diff : {mouth_count}')
print(f'arm diff : {arm_count}')
print(f'loc diff : {loc_count}')
print(f'size diff : {size_count}')

eye diff : 9
leg diff : 7
mouth diff : 25
arm diff : 8
loc diff : 6
size diff : 21


In [15]:
eye_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['eye_yn'].iloc[i] != person_pred_df['eye_yn'].iloc[i]:
        eye_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['eye_yn'].iloc[i], person_pred_df['eye_yn'].iloc[i]])

print(len(eye_wrong))
for i in eye_wrong:
    print(i)

9
['33_94_23072_person', 'y', 'n']
['33_95_23104_person', 'y', 'n']
['33_99_23212_person', 'y', 'no detect']
['3_11_23078_person', 'y', 'n']
['3_16_23058_person', 'y', 'n']
['52_216_23044_person', 'n', 'y']
['52_216_23050_person', 'n', 'y']
['52_216_23051_person', 'n', 'y']
['52_216_23053_person', 'n', 'y']


In [16]:
leg_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['leg_yn'].iloc[i] != person_pred_df['leg_yn'].iloc[i]:
        leg_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['leg_yn'].iloc[i], person_pred_df['leg_yn'].iloc[i]])

print(len(leg_wrong))
for i in leg_wrong:
    print(i)

7
['24_175_23020_person', 'y', 'n']
['33_93_23053_person', 'y', 'n']
['33_99_23212_person', 'n', 'no detect']
['38_107_23085_person', 'y', 'n']
['3_12_23102_person', 'y', 'n']
['50_227_23009_person', 'n', 'y']
['52_218_23113_person', 'n', 'y']


In [17]:
mouth_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['mouth_yn'].iloc[i] != person_pred_df['mouth_yn'].iloc[i]:
        mouth_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['mouth_yn'].iloc[i], person_pred_df['mouth_yn'].iloc[i]])

print(len(mouth_wrong))
for i in mouth_wrong:
    print(i)

25
['26_139_23009_person', 'y', 'n']
['33_91_23005_person', 'y', 'n']
['33_94_23072_person', 'y', 'n']
['33_95_23104_person', 'y', 'n']
['33_99_23206_person', 'y', 'n']
['33_99_23212_person', 'y', 'no detect']
['33_99_23220_person', 'y', 'n']
['38_104_23007_person', 'n', 'y']
['38_105_23025_person', 'y', 'n']
['3_11_23078_person', 'y', 'n']
['3_13_23106_person', 'y', 'n']
['3_13_23120_person', 'y', 'n']
['3_16_23049_person', 'y', 'n']
['3_16_23058_person', 'y', 'n']
['50_227_23006_person', 'n', 'y']
['50_227_23007_person', 'y', 'n']
['50_227_23013_person', 'n', 'y']
['50_228_23033_person', 'n', 'y']
['50_228_23040_person', 'n', 'y']
['50_229_23063_person', 'y', 'n']
['50_229_23071_person', 'y', 'n']
['50_231_23105_person', 'n', 'y']
['52_216_23048_person', 'n', 'y']
['52_217_23065_person', 'n', 'y']
['5_29_23031_person', 'n', 'y']


In [18]:
arm_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['arm_yn'].iloc[i] != person_pred_df['arm_yn'].iloc[i]:
        arm_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['arm_yn'].iloc[i], person_pred_df['arm_yn'].iloc[i]])

print(len(arm_wrong))
for i in arm_wrong:
    print(i)

8
['33_99_23212_person', 'n', 'no detect']
['3_13_23106_person', 'n', 'y']
['3_13_23108_person', 'n', 'y']
['3_16_23058_person', 'y', 'n']
['50_229_23071_person', 'y', 'n']
['50_230_23072_person', 'n', 'y']
['52_216_23048_person', 'y', 'n']
['52_218_23117_person', 'n', 'y']


In [21]:
loc_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['loc'].iloc[i] != person_pred_df['loc'].iloc[i]:
        loc_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['loc'].iloc[i], person_pred_df['loc'].iloc[i]])

print(len(loc_wrong))
for i in loc_wrong:
    print(i)

6
['24_175_23006_person', 'right', 'center']
['24_175_23007_person', 'center', 'right']
['24_175_23020_person', 'left', 'center']
['33_94_23085_person', 'left', 'center']
['33_99_23212_person', 'left', 'no detect']
['3_11_23078_person', 'right', 'center']


In [23]:
size_wrong = []
for i in range(len(person_pred_df)): 
    if person_ori_100_df['size'].iloc[i] != person_pred_df['size'].iloc[i]:
        size_wrong.append([person_pred_df['id'].iloc[i], person_ori_100_df['size'].iloc[i], person_pred_df['size'].iloc[i]])

print(len(size_wrong))
for i in size_wrong:
    print(i)

21
['24_175_23007_person', 'small', 'big']
['24_177_23044_person', 'small', 'middle']
['26_140_23025_person', 'middle', 'big']
['33_91_23005_person', 'middle', 'big']
['33_91_23022_person', 'big', 'middle']
['33_94_23084_person', 'middle', 'small']
['33_96_23130_person', 'big', 'middle']
['33_99_23212_person', 'middle', 'no detect']
['33_99_23220_person', 'big', 'middle']
['33_99_23225_person', 'small', 'middle']
['38_106_23053_person', 'middle', 'big']
['3_13_23120_person', 'small', 'big']
['47_198_23024_person', 'middle', 'small']
['50_227_23009_person', 'small', 'middle']
['50_229_23063_person', 'big', 'middle']
['50_230_23072_person', 'middle', 'small']
['50_230_23075_person', 'middle', 'big']
['50_230_23092_person', 'big', 'middle']
['52_216_23051_person', 'middle', 'small']
['52_216_23053_person', 'big', 'middle']
['52_219_23083_person', 'big', 'middle']
